In [6]:
import random
import math

In [7]:
def parse_input(file_path):
    """Parses the input file and extracts vehicle capacity and customer data."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Extract vehicle capacity
    vehicle_capacity = None
    customers = []

    for i, line in enumerate(lines):
        if "CAPACITY" in line:
            vehicle_capacity = int(lines[i + 1].split()[1])
        if "CUST NO." in line:
            start_idx = i + 1
            break

    # Parse customer data
    for line in lines[start_idx:]:
        parts = line.strip().split()
        if len(parts) >= 7:
            customer = {
                "id": int(parts[0]),
                "x": float(parts[1]),
                "y": float(parts[2]),
                "demand": int(parts[3]),
                "ready_time": int(parts[4]),
                "due_date": int(parts[5]),
                "service_time": int(parts[6]),
            }
            customers.append(customer)

    return vehicle_capacity, customers

In [8]:
def euclidean_distance(cust1, cust2):
    """Calculate Euclidean distance between two customers."""
    return math.sqrt((cust1["x"] - cust2["x"])**2 + (cust1["y"] - cust2["y"])**2)

In [9]:
def calculate_travel_time(customers):
    """Create a time matrix based on travel distance."""
    num_customers = len(customers)
    time_matrix = [[0] * num_customers for _ in range(num_customers)]
    for i in range(num_customers):
        for j in range(num_customers):
            time_matrix[i][j] = euclidean_distance(customers[i], customers[j])
    return time_matrix

In [12]:
class GeneticAlgorithmVRPTW:
    def __init__(self, vehicle_capacity, customers, population_size=50, generations=100, mutation_rate=0.1):
        self.vehicle_capacity = vehicle_capacity
        self.customers = customers
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.time_matrix = calculate_travel_time(customers)
        self.depot = customers[0]

    def initialize_population(self):
        """Generate the initial population of random routes."""
        population = []
        customer_ids = [c["id"] for c in self.customers[1:]]  # Exclude depot (id 0)
        for _ in range(self.population_size):
            random.shuffle(customer_ids)
            population.append(customer_ids[:])
        return population

    def fitness(self, individual):
        total_distance = 0
        current_vehicle_capacity = self.vehicle_capacity
        time = 0
        last_customer = self.depot
        penalty = 0

        for customer_id in individual:
            customer = self.customers[customer_id]
            distance = self.time_matrix[last_customer["id"]][customer["id"]]
            time += distance

            if time > customer["due_date"]:  # Late arrival penalty
                penalty += (time - customer["due_date"]) * 10  # Penalize lateness
            if customer["demand"] > current_vehicle_capacity:  # Over capacity penalty
                penalty += (customer["demand"] - current_vehicle_capacity) * 1000

            time = max(time, customer["ready_time"]) + customer["service_time"]
            current_vehicle_capacity -= customer["demand"]
            last_customer = customer

        total_distance += self.time_matrix[last_customer["id"]][self.depot["id"]]
        return total_distance + penalty


    def selection(self, population):
        """Select two individuals based on tournament selection."""
        tournament_size = 5
        selected = random.sample(population, tournament_size)
        selected.sort(key=self.fitness)
        return selected[0], selected[1]

    def crossover(self, parent1, parent2):
        """Perform ordered crossover between two parents."""
        size = len(parent1)
        start, end = sorted(random.sample(range(size), 2))
        child = [None] * size

        # Copy the segment from parent1
        child[start:end + 1] = parent1[start:end + 1]

        # Fill the rest from parent2
        current_idx = 0
        for gene in parent2:
            if gene not in child:
                while child[current_idx] is not None:
                    current_idx += 1
                child[current_idx] = gene

        return child

    def mutate(self, individual):
        """Perform mutation on an individual by swapping two cities."""
        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

    def optimize(self):
        """Run the genetic algorithm."""
        population = self.initialize_population()

        for generation in range(self.generations):
            new_population = []

            for _ in range(self.population_size // 2):
                parent1, parent2 = self.selection(population)
                child1 = self.crossover(parent1, parent2)
                child2 = self.crossover(parent2, parent1)

                self.mutate(child1)
                self.mutate(child2)

                new_population.append(child1)
                new_population.append(child2)

            population = new_population

            # Track the best solution
            best_individual = min(population, key=self.fitness)
            print(f"Generation {generation + 1}, Best Fitness: {self.fitness(best_individual)}")

        # Return the best solution
        return min(population, key=self.fitness)

    def decode_solution(self, solution):
        """Decode the solution into readable routes."""
        routes = []
        route = []
        current_vehicle_capacity = self.vehicle_capacity
        time = 0
        last_customer = self.depot

        for customer_id in solution:
            customer = self.customers[customer_id]
            distance = self.time_matrix[last_customer["id"]][customer["id"]]

            if (time + distance > customer["due_date"]) or (current_vehicle_capacity < customer["demand"]):
                # Start a new route
                routes.append(route)
                route = []
                current_vehicle_capacity = self.vehicle_capacity
                time = 0
                last_customer = self.depot

            time += distance
            time = max(time, customer["ready_time"]) + customer["service_time"]
            current_vehicle_capacity -= customer["demand"]
            route.append(customer_id)
            last_customer = customer

        # Add the last route
        if route:
            routes.append(route)

        return routes


In [13]:
if __name__ == "__main__":
    file_path = "c1/c101.txt"
    vehicle_capacity, customers = parse_input(file_path)

    ga_vrptw = GeneticAlgorithmVRPTW(vehicle_capacity, customers)
    best_solution = ga_vrptw.optimize()
    routes = ga_vrptw.decode_solution(best_solution)

    print("Optimized Routes:")
    for route in routes:
        print(" -> ".join(map(str, route)))

Generation 1, Best Fitness: 73041217.5241738
Generation 2, Best Fitness: 71044140.92238413
Generation 3, Best Fitness: 68841909.536848
Generation 4, Best Fitness: 68277974.86676855
Generation 5, Best Fitness: 68277974.86676855
Generation 6, Best Fitness: 68295399.55868317
Generation 7, Best Fitness: 67404022.7903942
Generation 8, Best Fitness: 66529469.641757585
Generation 9, Best Fitness: 65749683.91855095
Generation 10, Best Fitness: 64662670.659782
Generation 11, Best Fitness: 64170735.68746732
Generation 12, Best Fitness: 63412552.17713155
Generation 13, Best Fitness: 62121421.5115582
Generation 14, Best Fitness: 62432992.534762844
Generation 15, Best Fitness: 61785878.52500995
Generation 16, Best Fitness: 61584303.96823224
Generation 17, Best Fitness: 60572020.6074651
Generation 18, Best Fitness: 60658559.74637144
Generation 19, Best Fitness: 60334710.64807474
Generation 20, Best Fitness: 60043072.73333774
Generation 21, Best Fitness: 59625312.2560496
Generation 22, Best Fitness: 